In [1]:
import numpy as np
import cv2

import matplotlib.pyplot as plt



![](https://upload.wikimedia.org/wikipedia/commons/5/55/Opticfloweg.png)

![](data/out_optical_flow-min.gif)

In [2]:
def opticalFlow(img1, img2, window_size):
    
    fr2 = img2.copy()

    # Convert images to grayscale
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Define the motion model
    warp_mode = cv2.MOTION_TRANSLATION

    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY:
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else:
        warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Specify the number of iterations
    number_of_iterations = 5000

    # Specify the threshold of the increment
    # in the correlation coefficient between two iterations
    termination_eps = 1e-10

    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                number_of_iterations, termination_eps)

    # Run the ECC algorithm. The results are stored in warp_matrix.
    (cc, warp_matrix) = cv2.findTransformECC(img1, img2, warp_matrix,
                                             warp_mode, criteria)

    if warp_mode == cv2.MOTION_HOMOGRAPHY:
        # Use warpPerspective for Homography
        img2_aligned = cv2.warpPerspective(img2, warp_matrix,
                                           (img1.shape[1], img1.shape[0]),
                                             flags=cv2.INTER_LINEAR +   cv2.WARP_INVERSE_MAP)
    else:
        # Use warpAffine for Translation, Euclidean and Affine
        img2_aligned = cv2.warpAffine(img2, warp_matrix,
                                      (img1.shape[1], img1.shape[0]),
                                        flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);
        
    # calculate optical flow
    flow = cv2.calcOpticalFlowFarneback(img1, img2_aligned, None, 0.5, 3, window_size, 3, 5, 1.2, 0)
    
    # calculate magnitude and angle
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    
    # initialize the mask
    mask = np.zeros((720, 1280, 3))
    
    # set image hue according to the optical flow direction
    mask[..., 0] = angle * 180 / np.pi / 2
    
    # normalize magnitude
    mask[..., 1] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    
    # apply mask tresholding to the image
    mask[..., 2] = cv2.threshold(mask[..., 1], 25, 255, cv2.THRESH_BINARY)[1]
    
    # convert HSV to RGB
    # img2_al = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    
    return mask


i = 0 # counter for saving images

out = cv2.VideoWriter('data/output_mask_optical_flow.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 15, (1280, 720))

# load video
cap = cv2.VideoCapture('data/sample.mp4')

ret, frame1 = cap.read()

while True:
    
    ret, frame2 = cap.read()
    
    if not ret:
        break
    
    # optical flow
    img_mask = opticalFlow(frame1, frame2, 15)
    
    # update frame1
    frame1 = frame2
    
    # wait for key
    k = cv2.waitKey(30) & 0xff
    
    if k == 27:
        break
    
    # shwo frame
    cv2.imshow('frame', img_mask)
    
    # # convert into cv2 image convert to RGB
    img_mask = img_mask.clip(0, 1)
    img_mask = np.array(img_mask)
    
    # # save into a video 
    # out.write(img_mask)
    
    if i < 10:
        plt.imsave('data/hog_images/optical_00' + str(i) + '.png', img_mask)
    elif i < 100:
        plt.imsave('data/hog_images/optical_0' + str(i) + '.png', img_mask)
    elif i < 1000:
        plt.imsave('data/hog_images/optical_' + str(i) + '.png', img_mask)
        
    i += 1
    
    
cap.release()
cv2.destroyAllWindows()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [7]:
import glob 

path = sorted(glob.glob('data/hog_images/*.png'))

# merge all the images into a video
img_array = []
for filename in path:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    
out = cv2.VideoWriter('data/out_optical_flow.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 10, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [18]:
mask = np.zeros_like(frame1)
mask[..., 1].shape

(720, 1280)

In [25]:
# load mp4
video = cv2.VideoCapture('data/out_optical_flow.mp4')

# convert from mp4 to gif


(720, 1280)